In [19]:
from pandas import DataFrame
import pandas as pd
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from math import sqrt
import matplotlib
# be able to save images on server
matplotlib.use('Agg')
from matplotlib import pyplot
from matplotlib import pyplot as plt
import numpy

# date-time parsing function for loading the dataset
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	return df

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, yhat):
	new_row = [x for x in X] + [yhat]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# fit an MLP network to training data
def fit_model(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	model = Sequential()
	model.add(Dense(neurons, activation='relu', input_dim=X.shape[1]))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	model.fit(X, y, epochs=nb_epoch, batch_size=batch_size, verbose=0, shuffle=False)
	return model

# run a repeated experiment
def experiment(repeats, series, epochs, lag, neurons):
    # transform data to be stationary
    raw_values = series.values
    diff_values = difference(raw_values, 1)
    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, lag)
    supervised_values = supervised.values[lag:,:]
    # split data into train and test-sets
    train, test = supervised_values[0:-12], supervised_values[-12:]
    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)
    # run experiment
    error_scores = list()
    for r in range(repeats):
        # fit the model
        batch_size = 4
        train_trimmed = train_scaled[2:, :]
        model = fit_model(train_trimmed, batch_size, epochs, neurons)
        # forecast test dataset
        test_reshaped = test_scaled[:,0:-1]
        output = model.predict(test_reshaped, batch_size=batch_size)
        predictions = list()
        for i in range(len(output)):
            yhat = output[i,0]
            X = test_scaled[i, 0:-1]
            # invert scaling
            yhat = invert_scale(scaler, X, yhat)
            # invert differencing
            yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
            # store forecast
            predictions.append(yhat)
        # report performance
        print(predictions)
        print(raw_values[-12:])
#         pd.Series(raw_values[-12:]).plot()
#         pd.Series(predictions).plot()
#         plt.show()
        rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
        print('%d) Test RMSE: %.3f' % (r+1, rmse))
        error_scores.append(rmse)
    return error_scores

# load dataset
series = read_csv('./data/time_series_course_data/shampoo.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
# experiment
repeats = 1
results = DataFrame()
lag = 1
neurons = 1
# vary training epochs
epochs = [50]
for e in epochs:
	results[str(e)] = experiment(repeats, series, e, lag, neurons)
# summarize results
print(results.describe())
# save boxplot
results.boxplot()


[374.2492768697441, 371.6492768697441, 472.3492768697441, 290.24350754618644, 471.2492768697441, 419.1142620667815, 469.3492768697441, 607.449276869744, 360.1327213048935, 713.949276869744, 408.33370380699637, 613.249276869744]
[339.7 440.4 315.9 439.3 401.3 437.4 575.5 407.6 682.  475.3 581.3 646.9]
1) Test RMSE: 158.285
               50
count    1.000000
mean   158.285407
std           NaN
min    158.285407
25%    158.285407
50%    158.285407
75%    158.285407
max    158.285407


In [8]:
# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, lag)
supervised_values = supervised.values[lag:,:]
# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]
scaler, train_scaled, test_scaled = scale(train, test)

In [6]:
test_scaled

array([[ 0.26733207, -0.16644187],
       [-0.16644187,  0.39088211],
       [ 0.39088211, -0.82411654],
       [-0.82411654,  0.51335312],
       [ 0.51335312, -0.35743189],
       [-0.35743189,  0.04235231],
       [ 0.04235231,  0.59266253],
       [ 0.59266253, -1.05826814],
       [-1.05826814,  1.32802806],
       [ 1.32802806, -1.26760183],
       [-1.26760183,  0.41947667],
       [ 0.41947667,  0.20151066]])

In [11]:
train

array([[-120.1,   37.2],
       [  37.2,  -63.8],
       [ -63.8,   61. ],
       [  61. ,  -11.8],
       [ -11.8,   63.3],
       [  63.3,   -7.3],
       [  -7.3,  -31.7],
       [ -31.7,  -69.9],
       [ -69.9,  213.6],
       [ 213.6, -150.6],
       [-150.6,    8.4],
       [   8.4,  -44.8],
       [ -44.8,   60.6],
       [  60.6,   63.2],
       [  63.2,  -81.9],
       [ -81.9,   95.6],
       [  95.6,  -61. ],
       [ -61. ,   77.6],
       [  77.6,  -13.7],
       [ -13.7,  131.7],
       [ 131.7, -157.1],
       [-157.1,   77.8]])

In [12]:
test

array([[  77.8,   -2.6],
       [  -2.6,  100.7],
       [ 100.7, -124.5],
       [-124.5,  123.4],
       [ 123.4,  -38. ],
       [ -38. ,   36.1],
       [  36.1,  138.1],
       [ 138.1, -167.9],
       [-167.9,  274.4],
       [ 274.4, -206.7],
       [-206.7,  106. ],
       [ 106. ,   65.6]])

In [13]:
len(train), len(test)

(22, 12)